<img src="https://nlp.johnsnowlabs.com/assets/images/logo.png" width="180" height="50" style="float: left;">

## Sentiment Analysis Pipeline

This pipeline will be used to explain a number of important features of the Spark-NLP library; Sentence Detection, Tokenization, Spell Checking, and Sentiment Detection.
The idea is to start with natural language as could have been entered by a user, and get sentiment associated to it. Let's walk through each of the stages!


### Spark `2.4` and Spark NLP `???`

#### 1. Call necessary imports and set the resource path to read local data files

In [ ]:
#Imports
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import array_contains
from sparknlp.annotator import *
from pyspark.ml import Pipeline, PipelineModel

# location of pre-trained pipelines
resource_path= "../demo_pipelines/movies_sentiment"

#### 2. Load SparkSession if not already there

In [ ]:
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

spark = sparknlp.start()

#### 3. Load our predefined pipeline containing all the important annotators.

In [ ]:
pipeline = PretrainedPipeline("movies_sentiment_analysis")

#### 4. Create some user opinions for some movies, keep an eye on the spelling, we'll get back to that soon.

In [ ]:
testDocs = [
    "I felt so disapointed to see this very uninspired film. I recommend others to awoid this movie is not good.",
    "This was movie was amesome, everything was nice."]

In [ ]:
result = pipeline.annotate(testDocs)
[(r['sentence'], r['my_sda_scores']) for r in result]

 #### [Optional] - inspect intermmediate stages - spell checking

In [ ]:
[list(zip(r['token'],r['checked_token'])) for r in result]